Implementacion recuperada de:

https://github.com/shivamduseja/Deep-Summarization/blob/master/LSTM_with_attention.ipynb

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import sqlite3
dbfile = '/content/drive/MyDrive/Datos/NoticiasReforma2009.db'
import pandas as pd
import sqlite3
import sqlalchemy 

try:
    con = sqlite3.connect(dbfile)    
except Exception as e:
    print(e)

#Now in order to read in pandas dataframe we need to know table name
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Table Name : {cursor.fetchall()}")


df = pd.read_sql_query('SELECT * FROM Noticias2009', con)
con.close()


Table Name : [('Reforma2009',), ('Noticias2009',)]


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stopSpanish=set(stopwords.words('spanish'))
resumen=[]
for i in range (61035):
  text_tokens = word_tokenize(df['articulo'][i])
  tokens_without_sw = [word for word in text_tokens if not word in stopSpanish]
  tokens_without_sw = [word.lower() for word in tokens_without_sw if word.isalpha()]
  resumen.append(' '.join(tokens_without_sw))

In [ ]:
len(resumen)

61035

In [ ]:
data={'Noticia': resumen, 'Resumen': df['resumen'][:]}

In [ ]:
df1=pd.DataFrame(data)
df1.head()

,Noticia,Resumen
0,fallan reproductores zune microsoft en año nue...,Los reproductores de música Zune dejan de func...
1,san cadillase pasan trabajadores mientras hoy ...,Se pasan de trabajadores\n \nMientras hoy uste...
2,aleph cero materia antimateriaen principio luz...,"En el Principio eran la luz, la materia y la a..."
3,gaceta del ángel yo macho quantecedentes día a...,"ANTECEDENTES: el día de ayer, mientras confecc..."
4,circuito interiorunos tristes contentos los qu...,"Unos tristes, otros contentos\n \n \nLOS QUE a..."


In [ ]:
df1['Noticia']
cnt=0
for i in df1['Noticia']:
    if(len(i.split())<=250):
        cnt=cnt+1
print(cnt/len(df1['Noticia']))

0.8439256164495781


In [ ]:
Noticiasacotadas=[]
Resumenes=[]

Veo que toman datos con una longitud maxima, no se si sea lo mejor para el corpus

In [ ]:
for i in range(len(df1['Noticia'])):
  if(len(df1['Noticia'][i].split())<=250):
    Noticiasacotadas.append(df1['Noticia'][i])
    Resumenes.append(df1['Resumen'][i])

In [ ]:
len(Resumenes)
len(Noticiasacotadas)

51509

In [ ]:
dfinal=pd.DataFrame({'Noticias': Noticiasacotadas, 'Resumenes': Resumenes})
dfinal.head()

,Noticias,Resumenes
0,fallan reproductores zune microsoft en año nue...,Los reproductores de música Zune dejan de func...
1,circuito interiorunos tristes contentos los qu...,"Unos tristes, otros contentos\n \n \nLOS QUE a..."
2,la capillamuy movido naucalpan inicio año meno...,MUY MOVIDO estará en Naucalpan el inicio de añ...
3,plaza públicapor vacaciones columna granados c...,"Por vacaciones, la columna de Granados Chapa v..."
4,miradorplegaria un pedazo tierra posar planta ...,Plegaria\nUn pedazo de tierra para posar mi pl...


In [ ]:
#Generamos datos de pureba y entrenamiento
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dfinal['Noticias'],dfinal['Resumenes'],test_size=0.1,random_state=42,shuffle=True) 

#Creamos un vocabulario/tokens

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
def readLangs(text, summary, reverse=False):
    print("Reading lines...")
    
    # Split every line into pairs and normalize
    text=np.array(text)
    summary=np.array(summary)
    pairs = [[text[i],summary[i]] for i in range(len(text))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(summary)
        output_lang = Lang(text)
    else:
        input_lang = Lang(text)
        output_lang = Lang(summary)

    return input_lang, output_lang, pairs

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name,"--------------------", input_lang.n_words)
    #print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name,"--------------------", input_lang.n_words)
    #print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
input_lang, output_lang, pairs = prepareData(x_train, y_train , False)

Reading lines...
Read 46358 sentence pairs
Counting words...
Counted words:
['confía hugo remontadael técnico almería hugo sánchez dijo martes conjunto rojiblanco calidad ilusión conseguir remontada mallorca rival medirá miércoles vuelta octavos final copa perdió duelo ida apenas segundo partido mexicano frente sánchez confía jugar estadio mediterráneo haber anotado gol afuera casa permite pensar poder darle vuelta marcador cuando dicen eliminatoria difícil quito jugamos casa circunstancia ayudarnos además pesar encajar tres tantos mallorca conseguimos importancia todos muchas ganas intentarlo afirmó sánchez superar eliminatoria difícil traen resultado positivo nosotros hacer dos goles dejar portería cero calidad ilusión podemos conseguirlo juego almería sánchez reconoció aún restan muchas cosas mejorar ejemplo acciones balón parado aspecto ofensivo defensivo así tener mayor intensidad pero depende físico aún idóneo sostuvo'
 'derraman parejas miel sundanceel amor cine muchas veces van

In [ ]:
pairs[0]

['confía hugo remontadael técnico almería hugo sánchez dijo martes conjunto rojiblanco calidad ilusión conseguir remontada mallorca rival medirá miércoles vuelta octavos final copa perdió duelo ida apenas segundo partido mexicano frente sánchez confía jugar estadio mediterráneo haber anotado gol afuera casa permite pensar poder darle vuelta marcador cuando dicen eliminatoria difícil quito jugamos casa circunstancia ayudarnos además pesar encajar tres tantos mallorca conseguimos importancia todos muchas ganas intentarlo afirmó sánchez superar eliminatoria difícil traen resultado positivo nosotros hacer dos goles dejar portería cero calidad ilusión podemos conseguirlo juego almería sánchez reconoció aún restan muchas cosas mejorar ejemplo acciones balón parado aspecto ofensivo defensivo así tener mayor intensidad pero depende físico aún idóneo sostuvo',
 'El DT del Almería, Hugo Sánchez, dijo que su equipo tiene la capacidad para remontar en el juego de vuelta de la Copa del Rey ante Mal

In [ ]:
SOS_token = 0
EOS_token = 1

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F




In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Encoder: Lo veremos apenas

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.LSTM = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.LSTM(output, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),torch.zeros(1, 1, self.hidden_size, device=device)) 

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.LSTM = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.LSTM(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
MAX_LENGTH=250

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*2 , self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2 , self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.LSTM = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

        #print('Decoder --- atndecoder')

    def forward(self, input, hidden, encoder_outputs):
        #print("inside forward decoder")
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        #print("embedded size", embedded.size())
        #print(embedded[0].size()) #1,300
        #print(hidden[0].size()) # 1, 1, 300
        #print("diff or not")

        #temp = torch.cat((embedded[0], hidden[0]), 1)
        
        #print(temp)
        #print(temp.size())

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        #print("after getting attn weights softmax")
        #print(attn_weights.size())
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.LSTM(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
#teacher_forcing_ratio = 0.5
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    #print('bbbbbbb-->>> input length', input_length)

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        #print("priting before error")
        #print(encoder_output.size())
        #print(encoder_outputs.size())
        temp = encoder_output[0, 0]
        #print(temp)
        encoder_outputs[ei] = temp

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    #use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    #Without teacher forcing: use its own predictions as the next input

    #print('aaaaa-->>>')

    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
      decoder_input, decoder_hidden, encoder_outputs)


      topv, topi = decoder_output.topk(1)
      
      decoder_input = topi.squeeze().detach()  # detach from history as input
      
      loss += criterion(decoder_output, target_tensor[di])
      
      if decoder_input.item() == EOS_token:
             break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    print("Training....")
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        if iter% 1000 == 0:
            print(iter,"/",n_iters + 1)
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        input_length = input_tensor.size(0)
        if(input_length > 150):
          #print(input_length)
          continue
      
        loss = train(input_tensor, target_tensor, encoder,decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        #print('cccccc->>>>')

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=5):
    text=list()
    headline=list()
    pred_headline=list()    
    for i in range(n):
        pair = random.choice(pairs)
        
        if(len(pair[0].split())>=150):
          continue
        else:
          if(i%1000==0):
            print(i*100/n,"% complete")
          
          #print('>', pair[0])
          text.append(pair[0])
          #print('=', pair[1])
          headline.append(pair[1])
          output_words, attentions = evaluate(encoder, decoder, pair[0])
          output_sentence = ' '.join(output_words)
          pred_headline.append(output_sentence)
          #print('<', output_sentence)
          #print('')
    return(text,headline,pred_headline)

In [ ]:
#Entrenamineto:
hidden_size = 300
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 50000, print_every=1000)

Training....
1000 / 50001
2000 / 50001
9m 6s (- 218m 31s) (2000 4%) 5.7660
3000 / 50001
4000 / 50001
18m 20s (- 210m 58s) (4000 8%) 5.5724
5000 / 50001
23m 6s (- 207m 55s) (5000 10%) 2.7856
6000 / 50001
27m 36s (- 202m 26s) (6000 12%) 2.4453
7000 / 50001
32m 11s (- 197m 43s) (7000 14%) 2.5789
8000 / 50001
9000 / 50001
10000 / 50001
11000 / 50001
50m 38s (- 179m 32s) (11000 22%) 10.5467
12000 / 50001
55m 11s (- 174m 44s) (12000 24%) 2.6682
13000 / 50001
14000 / 50001
64m 29s (- 165m 49s) (14000 28%) 5.4044
15000 / 50001
69m 0s (- 161m 1s) (15000 30%) 2.4562
16000 / 50001
73m 55s (- 157m 4s) (16000 32%) 3.2138


In [ ]:
#Entrenamineto:
hidden_size = 300
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder2 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder2, attn_decoder2, 30000, print_every=1000)

Training....
1000 / 30001
4m 37s (- 134m 14s) (1000 3%) 2.7024
2000 / 30001
3000 / 30001
4000 / 30001
19m 10s (- 124m 38s) (4000 13%) 9.9269
5000 / 30001
24m 7s (- 120m 38s) (5000 16%) 3.2842
6000 / 30001
28m 47s (- 115m 9s) (6000 20%) 2.9561
7000 / 30001
33m 32s (- 110m 12s) (7000 23%) 2.8341
8000 / 30001
38m 17s (- 105m 18s) (8000 26%) 3.1246
9000 / 30001
42m 58s (- 100m 15s) (9000 30%) 2.8052
10000 / 30001
47m 29s (- 94m 59s) (10000 33%) 2.7465
11000 / 30001
52m 20s (- 90m 23s) (11000 36%) 2.5461
12000 / 30001
13000 / 30001
61m 29s (- 80m 24s) (13000 43%) 4.5962
14000 / 30001
15000 / 30001
16000 / 30001
17000 / 30001
80m 8s (- 61m 17s) (17000 56%) 10.5193
18000 / 30001
19000 / 30001
89m 32s (- 51m 50s) (19000 63%) 6.3307
20000 / 30001
94m 13s (- 47m 6s) (20000 66%) 2.9026
21000 / 30001
22000 / 30001
23000 / 30001
108m 8s (- 32m 54s) (23000 76%) 8.3107
24000 / 30001
112m 40s (- 28m 10s) (24000 80%) 2.8663
25000 / 30001
26000 / 30001
122m 1s (- 18m 46s) (26000 86%) 5.8828
27000 / 3000

In [ ]:
text,headline,pred_headline=evaluateRandomly(encoder1, attn_decoder1,15000)

0.0 % complete
6.666666666666667 % complete
20.0 % complete
26.666666666666668 % complete
33.333333333333336 % complete
40.0 % complete
53.333333333333336 % complete
60.0 % complete
80.0 % complete
93.33333333333333 % complete


In [ ]:
text,headline,pred_headline=evaluateRandomly(encoder2, attn_decoder2,15000)

0.0 % complete
20.0 % complete
40.0 % complete
53.333333333333336 % complete
60.0 % complete
66.66666666666667 % complete
86.66666666666667 % complete
93.33333333333333 % complete


In [ ]:
pred_df_LSTM=pd.DataFrame()
pred_df_LSTM['Noticias']=text
pred_df_LSTM['Resumenes']=headline
pred_df_LSTM['ResumenNew']=pred_headline

In [ ]:
pred_df_LSTM.head()

,Noticias,Resumenes,ResumenNew
0,elección francesacon divertidos discursos colo...,"Alumnas de la prepa de la UP, Campus Yaocalli,...",El de de a de en de el de de de de en de de de...
1,buscan tec jugar eulos itesm monterrey estado ...,"El coach del Tec CEM, Enrique Borda, viajó a E...",Un el de de de de en de de de de de de de de e...
2,confirma oasis cancelación tíbetel grupo britá...,Oasis confirmó que la anulación de los dos con...,Un de de a de el de en de de de de de de de de...
3,suman casos a morelosautoridades salud morelos...,Salud confirmó 16 nuevos casos de influenza A ...,Un de de de en de el de de de en de de de de d...
4,celebra soberanes salida medina mora el titula...,El titular de la CNDH aplaudió el relevo de Ed...,El el de de de en de de de de de de en de de d...


In [ ]:
pred_df_LSTM['ResumenNew'][1:10]

1    Un el de de de de en de de de de de de de de e...
2    Un de de a de el de en de de de de de de de de...
3    Un de de de en de el de de de en de de de de d...
4    El el de de de en de de de de de de en de de d...
5    Un de de a de en de de de de de de de de de de...
6    Un de de de en de el de de de de de en de de d...
7    Un de de de el en de de de de de de de de en d...
8    Un el de de de de en de de de de de de de de e...
9    El el de de de en a de de de de de de de de de...
Name: ResumenNew, dtype: object